In [155]:
import math
import pandas as pd
import numpy as np
import networkx as nx
# import community as community_louvain
import random
import igraph as ig
import leidenalg

nodes = pd.read_csv('synth3.attr', sep=' ', header=None, names=['node', 'attribute'])
edges = pd.read_csv('synth3.links', sep=' ', header=None, names=['source', 'target'])

n = len(nodes)
G = nx.Graph()

for _, row in nodes.iterrows():
    G.add_node(row['node'])

node_mapping = {node: i for i, node in enumerate(G.nodes())}

edges['source'] = edges['source'].map(node_mapping)
edges['target'] = edges['target'].map(node_mapping)

for _, row in edges.iterrows():
    G.add_edge(row['source'], row['target'])

matrix = nx.adjacency_matrix(G).todense()


In [156]:

G_ig = ig.Graph.from_networkx(G)
partition_leiden = leidenalg.find_partition(G_ig, leidenalg.ModularityVertexPartition)

R = nodes.groupby('attribute')['node'].apply(set)
AC = {0, 1, 2}
node_attr = {int(r['node']): r['attribute'] for _, r in nodes.iterrows()}

communities = {}
for node, cid in enumerate(partition_leiden.membership):
    if cid not in communities:
        communities[cid] = {}
    attr = node_attr.get(node)
    communities[cid][node] = attr

p = {i: {j: 0.01 for j in G.nodes()} for i in G.nodes()}

def modularity(G, partition):
    m = G.size(weight='weight')
    A = matrix
    degrees = np.array([G.degree(node) for node in G.nodes()])
    Q = 0.0
    for i in G.nodes():
        for j in G.nodes():
            if partition[i] == partition[j]:
                Q+=A[i, j]-(degrees[i]*degrees[j])/(2*m)

    return Q/(2*m)

print(partition_leiden)
modularity(G, partition_leiden.membership)

Clustering with 501 elements and 29 clusters
[ 0] 300, 301, 303, 304, 305, 306, 307, 308, 309, 311, 313, 314, 315, 316,
     317, 318, 320, 321, 322, 323, 325, 326, 327, 328, 329, 330, 331, 332,
     333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 344, 345, 346, 347,
     348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361,
     362, 363, 364, 365, 366, 367, 368, 369, 370, 372, 373, 374, 375, 376,
     377, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391,
     392, 393, 394, 396, 397, 400, 401, 403, 404, 405, 406, 407, 408, 409,
     410, 411, 412, 413, 415, 416, 417, 418, 419, 422, 492
[ 1] 6, 15, 16, 20, 26, 33, 38, 39, 45, 48, 54, 55, 57, 59, 62, 63, 81, 84,
     103, 111, 116, 119, 122, 138, 143, 144, 149, 181, 183, 186, 195, 202,
     205, 207, 228, 238, 241, 244, 245, 253, 255, 264, 266, 270, 272, 275,
     281, 292, 299
[ 2] 151, 161, 284, 424, 425, 426, 428, 429, 430, 431, 433, 434, 435, 436,
     437, 438, 439, 441, 443, 448, 449, 452, 453, 4

0.44452198333147647

In [157]:
def independent_cascade(G, S, p, num_simulations=100):
    total_activated = 0

    for _ in range(num_simulations):
        activated_nodes = set(S)
        newly_activated = set(S)

        while newly_activated:
            current_activated = set(newly_activated)
            newly_activated = set()

            for node in current_activated:
                for neighbor in set(G.neighbors(node)):
                    if neighbor not in activated_nodes:
                        if random.random() < p[node][neighbor]:
                            newly_activated.add(neighbor)
                            activated_nodes.add(neighbor)

        total_activated += len(activated_nodes)

    return total_activated / num_simulations

def linear_threshold(G, S, threshold=None, num_simulations=100):
    total_activated = 0

    for _ in range(num_simulations):
        activated = set(S)
        newly_activated = set(S)
        if threshold is None:
            node_threshold = {node: random.random() for node in G.nodes()}
        else:
            node_threshold = threshold.copy()
        influence = {node: 0.0 for node in G.nodes()}
        while newly_activated:
            current_activated = set(newly_activated)
            newly_activated = set()
            for node in current_activated:
                for neighbor in G.neighbors(node):
                    if neighbor not in activated:
                        weight = 1.0 / G.degree(neighbor)
                        influence[neighbor] += weight
            for node in G.nodes():
                if node not in activated and influence[node] >= node_threshold[node]:
                    activated.add(node)
                    newly_activated.add(node)

            total_activated += len(activated)

    return total_activated / num_simulations

def MF(G, S, R, p, num_simulations=100):
    min_influence_ratio = float('inf')

    for Ri in R:
        IGRiS = independent_cascade(G, S, p, num_simulations)
        # IGRiS = linear_threshold(G, S, None, num_simulations)
        influence_ratio = IGRiS / len(Ri)
        min_influence_ratio = min(min_influence_ratio, influence_ratio)

    return min_influence_ratio


In [158]:
subgraphs = {}
for i, Ri in enumerate(R):
    subgraph_nodes = list(Ri)
    subgraph = G.subgraph(subgraph_nodes)
    subgraphs[i] = subgraph

In [159]:
def DCV(G, S, R, p, subgraphs, num_simulations=100):
    total = 0
    i = 0

    for Ri in R:
        ki = max(1, (len(S) * len(Ri)) // len(G.nodes()))

        IGRiKi = independent_cascade(subgraphs[i], list(Ri)[:ki], p, num_simulations)
        IGRiS = independent_cascade(G, S, p, num_simulations)
        # IGRiKi = linear_threshold(subgraphs[i], list(Ri)[:ki], None, num_simulations)
        # IGRiS = linear_threshold(G, S, None, num_simulations)
        value = (IGRiKi - IGRiS) / (IGRiKi + 1e-10)
        total += max(value, 0)
        i += 1

    return total / len(R)


def F(G, S, R, p, subgraphs, lamda=0.5, num_simulations=100):
    mf_val = MF(G, S, R, p, num_simulations)
    dcv_val = DCV(G, S, R, p, subgraphs, num_simulations)
    return lamda * mf_val - (1 - lamda) * dcv_val


def CAj(communities, attributeJ):
    total = 0
    for community_id, community_data in communities.items():
        for node, attribute in community_data.items():
            if attribute == attributeJ:
                total += 1
    return total


def uj(nodes, attributeJ, communities):
    Aj = 0
    for _, row in nodes.iterrows():
        if row['attribute'] == attributeJ:
            Aj += 1

    if Aj == 0:
        return 0

    caj = CAj(communities, attributeJ)
    return math.exp(-caj / Aj)


def SCi(community_id, communities, nodes, AC):
    community_data = communities[community_id]
    community_size = len(community_data)

    total_urgency = 0
    for attribute in AC:
        total_urgency += uj(nodes, attribute, communities)

    return community_size * total_urgency


def pCi(community_id, communities, nodes, AC):
    total_sc = 0
    for cid in communities.keys():
        total_sc += SCi(cid, communities, nodes, AC)

    if total_sc == 0:
        return 1 / len(communities)

    return SCi(community_id, communities, nodes, AC) / total_sc

In [160]:
def SN(G, max_iter=100, d=0.85):
    nodes = list(G.nodes())
    n = len(nodes)

    out_degree = {node: G.degree(node) for node in nodes}
    neighbors = {node: list(G.neighbors(node)) for node in nodes}

    sn = {node: 1 / n for node in nodes}

    for _ in range(max_iter):
        new_sn = {}
        for node in nodes:
            sum_tmp = 0
            for neighbor in neighbors[node]:
                if out_degree[neighbor] > 0:
                    sum_tmp += sn[neighbor] / out_degree[neighbor]
            new_sn[node] = (1 - d) / n + d * sum_tmp
        sn = new_sn

    return sn

def pvi(node, community_id, communities, nodeScore):
    community_nodes = list(communities[community_id].keys())

    if node not in community_nodes:
        return 0

    total_score = sum(nodeScore.get(n, 0) for n in community_nodes)

    if total_score == 0:
        return 1 / len(community_nodes)

    return nodeScore.get(node, 0) / total_score

# Tính điểm node
nodeScore = SN(G)
nodeScore

{1: 0.002627761563351709,
 2: 0.0024011562564588626,
 3: 0.0015513873033516105,
 4: 0.0019575723462450577,
 5: 0.0019582578917088284,
 6: 0.0034257534633546183,
 7: 0.002152014475480675,
 8: 0.0037454802470050902,
 9: 0.0012546684245442633,
 10: 0.0032118723701003516,
 11: 0.002510112623524865,
 12: 0.0029217356235377054,
 13: 0.0015389063470127487,
 14: 0.002384405663767524,
 15: 0.0017506041472942098,
 16: 0.001438754930510006,
 17: 0.0024784601380637406,
 18: 0.002440926701805446,
 19: 0.001989294327723723,
 20: 0.001702290393711882,
 21: 0.002935481831777859,
 22: 0.0023648953382914596,
 23: 0.0019939481931684562,
 24: 0.0016786781419472585,
 25: 0.002766851014128461,
 26: 0.001758681858154049,
 27: 0.0030825817063930743,
 28: 0.0030913548619962063,
 29: 0.0015794895003498487,
 30: 0.0024739327851452503,
 31: 0.0009966940393142814,
 32: 0.0014925226448873285,
 33: 0.0030506658210302687,
 34: 0.0017802760338463596,
 35: 0.0019443298101384674,
 36: 0.00246523227629523,
 37: 0.0022753

In [161]:
pop_size = 10
k = 40
max_generations = 150
crossover_prob = 0.6
mutation_prob = 0.1

In [162]:
def initialize_population(G, pop_size, k, communities, nodeScore, nodes, AC):
    population = []

    for _ in range(pop_size):
        individual = set()
        community_counter = {cid: 0 for cid in communities.keys()}
        community_scores = {}
        for cid in communities.keys():
            community_scores[cid] = SCi(cid, communities, nodes, AC)

        for _ in range(k):
            total_score = sum(community_scores.values())
            if total_score == 0:
                selected_cid = random.choice(list(communities.keys()))
            else:
                rand_val = random.random() * total_score
                cumulative = 0
                for cid, score in community_scores.items():
                    cumulative += score
                    if cumulative >= rand_val:
                        selected_cid = cid
                        break

            community_counter[selected_cid] += 1

            community_scores[selected_cid] = SCi(selected_cid, communities, nodes, AC) * 0.5

        for cid, count in community_counter.items():
            if count>0:
                community_nodes = list(communities[cid].keys())
                sorted_nodes = sorted(community_nodes, key=lambda x: nodeScore.get(x, 0), reverse=True)
                selected_nodes = sorted_nodes[:count]
                individual.update(selected_nodes)
        while len(individual) < k:
            random_node = random.choice(list(G.nodes()))
            individual.add(random_node)

        individual = set(list(individual)[:k])
        population.append(list(individual))

    return population

In [163]:
def crossover(parent1, parent2, communities, nodeScore, nodes, AC, crossover_prob):
    if random.random() > crossover_prob:
        return parent1.copy(), parent2.copy()

    child1 = []
    child2 = []

    for i in range(len(parent1)):
        if random.random() < 0.5:
            child1.append(parent1[i])
            child2.append(parent2[i])
        else:
            child1.append(parent2[i])
            child2.append(parent1[i])
    child1 = list(set(child1))
    child2 = list(set(child2))

    while len(child1) < len(parent1):
        community_scores = {}
        for cid in communities.keys():
            community_scores[cid] = SCi(cid, communities, nodes, AC)
        total_score = sum(community_scores.values())
        if total_score == 0:
            selected_cid = random.choice(list(communities.keys()))
        else:
            rand_val = random.random() * total_score
            cumulative = 0
            for cid, score in community_scores.items():
                cumulative += score
                if cumulative >= rand_val:
                    selected_cid = cid
                    break

        community_nodes = list(communities[selected_cid].keys())
        if community_nodes:
            node_probs = {}
            for node in community_nodes:
                if node not in child1:
                    node_probs[node] = pvi(node, selected_cid, communities, nodeScore)

            if node_probs:
                total_prob = sum(node_probs.values())
                if total_prob > 0:
                    rand_val = random.random() * total_prob
                    cumulative = 0
                    for node, prob in node_probs.items():
                        cumulative += prob
                        if cumulative >= rand_val:
                            child1.append(node)
                            break

    while len(child2) < len(parent2):
        community_scores = {}
        for cid in communities.keys():
            community_scores[cid] = SCi(cid, communities, nodes, AC)

        total_score = sum(community_scores.values())
        if total_score == 0:
            selected_cid = random.choice(list(communities.keys()))
        else:
            rand_val = random.random() * total_score
            cumulative = 0
            for cid, score in community_scores.items():
                cumulative += score
                if cumulative >= rand_val:
                    selected_cid = cid
                    break

        community_nodes = list(communities[selected_cid].keys())
        if community_nodes:
            node_probs = {}
            for node in community_nodes:
                if node not in child2:
                    node_probs[node] = pvi(node, selected_cid, communities, nodeScore)

            if node_probs:
                total_prob = sum(node_probs.values())
                if total_prob > 0:
                    rand_val = random.random() * total_prob
                    cumulative = 0
                    for node, prob in node_probs.items():
                        cumulative += prob
                        if cumulative >= rand_val:
                            child2.append(node)
                            break

    return child1[:len(parent1)], child2[:len(parent2)]

In [164]:
def mutate(individual, communities, nodeScore, nodes, AC, mutation_prob):
    mutated = individual.copy()

    for i in range(len(mutated)):
        if random.random() < mutation_prob:
            community_scores = {}
            for cid in communities.keys():
                community_scores[cid] = SCi(cid, communities, nodes, AC)

            total_score = sum(community_scores.values())
            if total_score == 0:
                selected_cid = random.choice(list(communities.keys()))
            else:
                rand_val = random.random() * total_score
                cumulative = 0
                for cid, score in community_scores.items():
                    cumulative += score
                    if cumulative >= rand_val:
                        selected_cid = cid
                        break
            community_nodes = list(communities[selected_cid].keys())
            if community_nodes:
                node_probs = {}
                for node in community_nodes:
                    if node not in mutated:
                        node_probs[node] = pvi(node, selected_cid, communities, nodeScore)
                if node_probs:
                    total_prob = sum(node_probs.values())
                    if total_prob > 0:
                        rand_val = random.random() * total_prob
                        cumulative = 0
                        for node, prob in node_probs.items():
                            cumulative += prob
                            if cumulative >= rand_val:
                                mutated[i] = node
                                break

    return mutated

In [165]:
def CEA_FIM(G, R, p, subgraphs, pop_size, k, max_generations, crossover_prob, mutation_prob, communities, nodes, AC):
    nodeScore = SN(G)

    population = initialize_population(G, pop_size, k, communities, nodeScore, nodes, AC)

    best_solution = None
    best_fitness = -float('inf')

    for generation in range(max_generations):
        fitness_scores = []
        for individual in population:
            fitness = F(G, individual, R, p, subgraphs)
            fitness_scores.append(fitness)

            if fitness > best_fitness:
                best_fitness = fitness
                best_solution = individual.copy()

        sorted_population = [x for _, x in sorted(zip(fitness_scores, population), reverse=True)]

        new_population = []

        for i in range(0, pop_size, 2):
            if i + 1 < pop_size:
                parent1 = sorted_population[i]
                parent2 = sorted_population[pop_size - i - 1]

                child1, child2 = crossover(parent1, parent2, communities, nodeScore, nodes, AC, crossover_prob)
                new_population.extend([child1, child2])
            else:
                new_population.append(sorted_population[i])

        for i in range(len(new_population)):
            new_population[i] = mutate(new_population[i], communities, nodeScore, nodes, AC, mutation_prob)
        new_fitness = [F(G, ind, R, p, subgraphs) for ind in new_population]
        combined_population = population + new_population
        combined_fitness = fitness_scores + new_fitness

        selected_indices = sorted(range(len(combined_fitness)), key=lambda i: combined_fitness[i], reverse=True)[
            :pop_size]
        population = [combined_population[i] for i in selected_indices]

        print(f"Generation {generation + 1}, Best Fitness: {best_fitness}")

    return best_solution, best_fitness

In [166]:
best_seed_set, best_fitness = CEA_FIM(
    G, R, p, subgraphs, pop_size, k, max_generations,
    crossover_prob, mutation_prob, communities, nodes, AC
)
print("Best seed set:", best_seed_set)
print("Best fitness:", best_fitness)
mf_value = MF(G, best_seed_set, R, p)
dcv_value = DCV(G, best_seed_set, R, p, subgraphs)
print("MF:", mf_value)
print("DCV:", dcv_value)

Generation 1, Best Fitness: 0.07335
Generation 2, Best Fitness: 0.07335
Generation 3, Best Fitness: 0.07335
Generation 4, Best Fitness: 0.07335
Generation 5, Best Fitness: 0.07393333333333334
Generation 6, Best Fitness: 0.07393333333333334
Generation 7, Best Fitness: 0.07393333333333334
Generation 8, Best Fitness: 0.07393333333333334
Generation 9, Best Fitness: 0.07393333333333334
Generation 10, Best Fitness: 0.07393333333333334
Generation 11, Best Fitness: 0.07393333333333334
Generation 12, Best Fitness: 0.07393333333333334
Generation 13, Best Fitness: 0.07393333333333334
Generation 14, Best Fitness: 0.07393333333333334
Generation 15, Best Fitness: 0.07393333333333334
Generation 16, Best Fitness: 0.07393333333333334
Generation 17, Best Fitness: 0.07393333333333334
Generation 18, Best Fitness: 0.07393333333333334
Generation 19, Best Fitness: 0.07393333333333334
Generation 20, Best Fitness: 0.07393333333333334
Generation 21, Best Fitness: 0.07393333333333334
Generation 22, Best Fitness: